# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096833


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:22,  2.84s/it]

Rendering models:  10%|▉         | 3/31 [00:07<01:12,  2.60s/it]

Rendering models:  13%|█▎        | 4/31 [00:08<00:52,  1.94s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:38,  1.48s/it]

...

Rendering models:  87%|████████▋ | 27/31 [00:17<00:01,  2.07it/s]

Rendering models:  94%|█████████▎| 29/31 [00:17<00:00,  2.60it/s]

Rendering models:  97%|█████████▋| 30/31 [00:17<00:00,  2.79it/s]

Rendering models: 100%|██████████| 31/31 [00:18<00:00,  2.71it/s]

not-logged-in-3b0b929f4475fd276561     0.002304
not-logged-in-3b0b929f4475fd276561     0.000725
not-logged-in-461bf2f260fc99e4866e     0.001778
Jorgearcelarreta                       0.000774
ElisabethB                             0.000788
not-logged-in-c6917d9b19f41e6c2995     0.001506
awright5                               0.003091
AaronWallage117                        0.025605
AmeenDeen                              0.002208
ferlin1998                             0.001426
not-logged-in-9a45911eb71184fe7c7f     0.001452
not-logged-in-5f30b2bb9b7f9d49da41     0.008564
not-logged-in-5f30b2bb9b7f9d49da41    66.768838
DarioB                                 0.000730
Planetme                               0.001158
Dpwillet                               0.000944
dthomas02                              0.002439
helios1235                             0.000677
Lavadude                               0.024182
Mr_DMan                                0.001135
BrowardDaniel                          0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())